In [49]:
from jbfg.pandas_utils import * 
import pandas as pd
from pandasql import sqldf

In [50]:
#  sql 사용 
pysqldf = lambda q: sqldf(q,globals())

In [51]:
schema='l2_jbbk_extnl'

In [52]:
# 상품정보 테이블 
products_info_sql='SELECT * FROM products_info'

# 5,859명 고객들 과거 구매 이력  
train_x_past_sql='SELECT * FROM train_x_past'

# 5,859명 고객들 최근 주문성보
train_x_current_sql = 'SELECT * FROM train_x_current'  

# 5,859명 고객들 최근 주문상품(train_x_current 각 row에 매핑되는 상품정보)
train_y_sql = 'SELECT * FROM train_y'  

# 4,141명 고객들 과거 구매 이력  
test_x_past_sql='SELECT * FROM test_x_past'

# 4,141명 고객들 최근 주문성보
test_x_current_sql='SELECT * FROM test_x_current'

In [61]:
# 데이터 불러오기 
products_info = get_df(products_info_sql, schema) 
train_x_past = get_df(train_x_past_sql, schema) 
train_x_current = get_df(train_x_current_sql, schema)  
train_y = get_df(train_y_sql, schema) 

# 분석용데이터
test_x_past = get_df(test_x_past_sql, schema)  
test_x_current = get_df(test_x_current_sql, schema)  

In [66]:
# train 및 products 파일크기 확인
print("products_info",products_info.shape)
print("train_x_past",train_x_past.shape)
print("train_x_current",train_x_current.shape)
print("train_y",train_y.shape)


products_info (100, 6)
train_x_past (1253972, 9)
train_x_current (24424, 8)
train_y (24424, 1)


In [73]:
train_x_past.describe()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
count,1253972,1253972,1253972,1253972,1253972,1253972,1253972,1253972,1253972
unique,341678,5859,99,7,24,32,93,2,100
top,2538975,150840,1,0,10,7.0,1,1,24852
freq,26,795,21686,233220,111896,166085,133657,1083774,74686


In [99]:
# 결측치 확인 
print("★products_info 결측치확인:\n",products_info.isnull().sum())
print("★train_x_past 결측치확인:\n",train_x_past.isnull().sum())
print("★train_x_current 결측치확인:\n",train_x_current.isnull().sum())
print("★train_y 결측치확인:\n",train_y.isnull().sum())

★products_info 결측치확인:
 product_id       0
name             0
category_id      0
division_id      0
category_name    0
division_name    0
dtype: int64
★train_x_past 결측치확인:
 order_id          0
cstno             0
order_number      0
week_day          0
order_hour        0
order_interval    0
add_order         0
is_reordered      0
product_id        0
dtype: int64
★train_x_current 결측치확인:
 order_id          0
cstno             0
order_number      0
week_day          0
order_hour        0
order_interval    0
add_order         0
is_reordered      0
dtype: int64
★train_y 결측치확인:
 product_id    0
dtype: int64


In [100]:
#product_in : 상품id / name : 상품명 / category_id 분류ID / division_id 담당부서아이디 / category_name 분류명 / division_name 담당부서명
products_info.tail()

,product_id,name,category_id,division_id,category_name,division_name
95,47209,Organic Hass Avocado,24,4,fresh fruits,produce
96,21903,Organic Baby Spinach,123,4,packaged vegetables fruits,produce
97,21137,Organic Strawberries,24,4,fresh fruits,produce
98,13176,Bag of Organic Bananas,24,4,fresh fruits,produce
99,24852,Banana,24,4,fresh fruits,produce


In [37]:
# Order_id : 주문 id / cstno : 고객id / order_number : 고객별 주문서순번(구매일자별순번)  / week_day : 구매요일 / order_hour : 구매시각 
# / order_interbal : 직전구매경과일수 / add_order : 주문서내 담긴 상품의 순번 / is_reordered : 재구매여부 / product_id 상품id  
train_x_past.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
1253967,1902783,163819,47,0,19,13.0,2,1,24852
1253968,1969590,163819,42,1,15,6.0,3,1,24852
1253969,2549215,163819,9,1,11,7.0,1,1,24852
1253970,3103211,163819,39,1,14,7.0,3,1,24852
1253971,3207262,163819,8,1,12,7.0,1,1,24852


In [58]:
train_x_current.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered
24419,3184000,174046,100,5,7,2.0,12,1
24420,1754722,6710,100,5,11,3.0,21,1
24421,1076973,150840,100,0,13,0.0,18,1
24422,1247049,99707,100,1,13,3.0,6,1
24423,1775787,201038,100,5,9,2.0,1,1


1

In [57]:
train_y.count()

product_id    24424
dtype: int64

In [30]:
pysqldf("""
select * from df_products_info  order by 1
""")

,product_id,name,category_id,division_id,category_name,division_name
0,10749,Organic Red Bell Pepper,83,4,fresh vegetables,produce
1,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs
2,12341,Hass Avocados,32,4,packaged produce,produce
3,13176,Bag of Organic Bananas,24,4,fresh fruits,produce
4,15290,Orange Bell Pepper,83,4,fresh vegetables,produce
...,...,...,...,...,...,...
95,8424,Broccoli Crown,83,4,fresh vegetables,produce
96,8518,Organic Red Onion,83,4,fresh vegetables,produce
97,9076,Blueberries,116,1,frozen produce,frozen
98,9387,Granny Smith Apples,24,4,fresh fruits,produce


,product_id,name,category_id,division_id,category_name,division_name
95,47209,Organic Hass Avocado,24,4,fresh fruits,produce
96,21903,Organic Baby Spinach,123,4,packaged vegetables fruits,produce
97,21137,Organic Strawberries,24,4,fresh fruits,produce
98,13176,Bag of Organic Bananas,24,4,fresh fruits,produce
99,24852,Banana,24,4,fresh fruits,produce
